In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

def num_connected_components(A, tol=1e-8, thresh = 0.98):
    # eps = 1e-8
    A = A - A.mean(dim=1,keepdim=True)
    A_norm = A / (A.norm(dim=1, keepdim=True) + tol)
    Corr = A_norm @ A_norm.T
    # print(Corr[:2,:2], Corr.shape)
    Corr.fill_diagonal_(0)
    Corr = Corr.abs()
    Adj = (Corr>0.98
          ).float()
    # Compute the degree vector and degree matrix D
    degrees = torch.sum(Adj, dim=1)
    D = torch.diag(degrees)
    # Compute the Laplacian L = D - Adj
    L = D - Adj
    # Compute eigenvalues of L. Since L is symmetric, use eigvalsh.
    eigenvalues = torch.linalg.eigvalsh(L)
    # Count the number of eigenvalues that are close to zero.
    num_components = torch.sum(eigenvalues < tol).item()
    return num_components


def compute_effective_rank(activation_matrix, eps=1e-12):
    """
    Compute the effective rank of an activation matrix in a numerically stable manner.
    
    activation_matrix: Tensor of shape (batch_size, feature_dim)
    eps: Small constant to prevent division by zero or log(0)
    
    Returns: effective rank (float)
    """
    # Use double precision for stability
    act = activation_matrix.double()
    # Compute SVD
    U, S, V = torch.linalg.svd(act, full_matrices=False)
    S_sum = S.sum() + eps  # Avoid division by zero
    p = S / S_sum         # Normalized singular values
    # Clamp probabilities to avoid log(0)
    p_clamped = p.clamp(min=eps)
    # Compute entropy and effective rank
    entropy = -(p * torch.log(p_clamped)).sum()
    eff_rank = torch.exp(entropy)
    return eff_rank.item()

# ---------------------
# Define a Wide, Deep MLP with Hooks to Record Activations
# ---------------------
class WideDeepMLP(nn.Module):
    def __init__(self, input_dim=3*32*32, hidden_dim=1000, num_layers=10, num_classes=10):
        super(WideDeepMLP, self).__init__()
        self.layers = nn.ModuleList()
        # First Linear Layer + ReLU
        self.layers.append(nn.Linear(input_dim, hidden_dim, bias=False))
        self.layers.append(nn.Tanh())
        # Additional hidden layers: each has a Linear layer followed by ReLU
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim, bias=False))
            self.layers.append(nn.Tanh())
        # Final classifier layer (not included in activation collection)
        self.layers.append ( nn.Linear(hidden_dim, num_classes, bias=False) ) 
        # Dictionary to store activations (from each hidden Linear layer output)
        self.activations = {}
        # Register hooks on each Linear layer in self.layers (skip ReLU modules)
        layer_idx = 0
        for module in self.layers:
            # store pre-activations (after applying linear )
            if not isinstance(module, nn.Linear):
                module.register_forward_hook(self._get_activation_hook(layer_idx))
                layer_idx += 1

        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0, std=(1.0/0.39)**0.5/layer.weight.shape[0]**0.5)

    def _get_activation_hook(self, idx):
        def hook(module, input, output):
            self.activations[f"layer_{idx}"] = output.detach()
        return hook

    def forward(self, x):
        # Flatten the input
        x = x.view(x.size(0), -1)
        for layer in self.layers:
            x = layer(x)
        return x

# ---------------------
# Data Preparation
# ---------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_set   = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=512, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_set, batch_size=6000, shuffle=False, num_workers=2, drop_last=True)

# ---------------------
# Initialize Model, Loss, Optimizer
# ---------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WideDeepMLP(num_layers=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
th = 0.999 # threshold for counting the connected components 

# ---------------------
# Training Loop`
# ---------------------
num_epochs = 1
for epoch in range(num_epochs):
    # Training phase, skip epoch 0, only report validation metrics 
    if epoch==0:
        avg_train_loss = 0
    if epoch > 0:
        model.train()
        train_loss_total = 0.0
        num_train_batches = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss_total += loss.item()
            num_train_batches += 1
        avg_train_loss = train_loss_total / num_train_batches

    # Validation phase: compute average loss on validation set
    model.eval()
    val_loss_total = 0.0
    num_val_batches = 0
    # Also, capture activations from one batch for effective rank
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()
            num_val_batches += 1
            # For effective rank metrics, only use the first batch
            if num_val_batches == 1:
                val_batch_activations = {k: v for k, v in model.activations.items()}
    avg_val_loss = val_loss_total / num_val_batches
    print(f'epoch {epoch} CC stats using threshold = {th:.3f}')
    for k,A in model.activations.items():
        print(f"layer {k} feature dim = {A.shape[1]} # of connected components: {num_connected_components(A.T,thresh=th)}")

    # Compute effective rank for each recorded hidden layer on the first validation batch
    erank_dict = {}
    for layer_name, act in val_batch_activations.items():
        act_matrix = act.view(act.size(0), -1)
        erank_dict[layer_name] = compute_effective_rank(act_matrix)

    # Report training loss, validation loss, and effective rank per layer for this epoch
    erank_str = ", ".join([f"{k}: {v:.2f}" for k, v in erank_dict.items()])
    print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f} | Effective Rank per layer: {erank_str}")


Files already downloaded and verified
Files already downloaded and verified
epoch 0 CC stats using threshold = 0.999
layer layer_0 feature dim = 1000 # of connected components: 1000
layer layer_1 feature dim = 1000 # of connected components: 1000
layer layer_2 feature dim = 1000 # of connected components: 1000
layer layer_3 feature dim = 1000 # of connected components: 1000
layer layer_4 feature dim = 1000 # of connected components: 1000
layer layer_5 feature dim = 1000 # of connected components: 1000
layer layer_6 feature dim = 1000 # of connected components: 1000
Epoch 1: Train Loss: 0.0000, Val Loss: 15.2584 | Effective Rank per layer: layer_0: 658.19, layer_1: 671.23, layer_2: 675.84, layer_3: 678.75, layer_4: 681.63, layer_5: 684.59, layer_6: 687.17


In [3]:
def compute_per_sample_gradients(model, data_loader, criterion, device, max_samples=1000):
    """
    Compute gradients with memory optimization.
    
    Args:
        model (nn.Module): The neural network model
        data_loader (DataLoader): Data loader
        criterion (callable): Loss function
        device (torch.device): Device for computation
        max_samples (int): Maximum number of samples to process
    """
    model.eval()
    per_sample_grads = []
    samples_processed = 0
    
    for images, labels in data_loader:
        batch_size = images.size(0)
        
        # Process smaller mini-batches
        mini_batch_size = 10  # Adjust this based on your GPU memory
        for i in range(0, batch_size, mini_batch_size):
            if samples_processed >= max_samples:
                break
                
            end_idx = min(i + mini_batch_size, batch_size)
            mini_images = images[i:end_idx].to(device)
            mini_labels = labels[i:end_idx].to(device)
            
            for j in range(mini_images.size(0)):
                # Process single sample
                sample_image = mini_images[j:j+1]
                sample_label = mini_labels[j:j+1]
                
                outputs = model(sample_image)
                loss = criterion(outputs, sample_label)
                
                # Compute and store gradients
                grads = torch.autograd.grad(loss, model.parameters(), create_graph=False)
                
                sample_grad = {}
                for (name, _), grad in zip(model.named_parameters(), grads):
                    # Store gradients as CPU tensors to save GPU memory
                    sample_grad[name] = grad.detach().cpu()
                
                per_sample_grads.append(sample_grad)
                samples_processed += 1
                
                # Clear some memory
                del grads, outputs, loss
                torch.cuda.empty_cache()
            
            # Clear mini-batch tensors
            del mini_images, mini_labels
            torch.cuda.empty_cache()
            
            if samples_processed >= max_samples:
                break
                
        if samples_processed >= max_samples:
            break
    
    return per_sample_grads

# Usage example:
max_samples = 100  # Adjust based on your needs
sample_gradients = compute_per_sample_gradients(model, train_loader, criterion, device, max_samples=max_samples)

# To analyze gradients:
print(f"Processed {len(sample_gradients)} samples")
first_sample_grads = sample_gradients[0]
for param_name, grad in first_sample_grads.items():
    print(f"{param_name}: {grad.shape}")

Processed 100 samples
layers.0.weight: torch.Size([1000, 3072])
layers.2.weight: torch.Size([1000, 1000])
layers.4.weight: torch.Size([1000, 1000])
layers.6.weight: torch.Size([1000, 1000])
layers.8.weight: torch.Size([1000, 1000])
layers.10.weight: torch.Size([1000, 1000])
layers.12.weight: torch.Size([1000, 1000])
layers.14.weight: torch.Size([10, 1000])


In [7]:
keys = sample_gradients[0].keys()
N = len(sample_gradients)
ntk = {k:torch.zeros((N,N)) for k in keys}

100

In [8]:
torch.zeros((2,2))

tensor([[0., 0.],
        [0., 0.]])